In [61]:
import string 
import random

# Make a random name for each page

In [62]:
cc = string.ascii_lowercase

def name():
    return 'X_'+ ''.join(random.choices(cc,k=3)) + '.html'

In [65]:
name()

'X_dti.html'

# Make a random adjacency matrix

we want the graph to be connected so 

- we start  with a [cycle](https://en.wikipedia.org/wiki/Cycle_graph#:~:text=In%20graph%20theory%2C%20a%20cycle,vertices%20is%20called%20Cn.)
- add a random matrix of 1,0
- clamp the values to 1,0

the matrix is not necessarily symmetric.

In [81]:
import numpy as np

n = 7
M = np.identity(n,dtype =np.int)
# this slides everything over by 1
M = np.roll(M,1, axis=1)

M = M + np.random.randint(2,size=(n,n))
M[M>1] = 1

# don't allow 
for k, r in enumerate(M):
    r[k] = 0


In [82]:
M

array([[0, 1, 1, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 1, 1, 0],
       [0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 1, 1],
       [1, 0, 0, 0, 1, 0, 1],
       [1, 1, 0, 1, 0, 0, 0]])

In [83]:
sites = [name() for k in range(n)]
lks = dict(zip(sites,M))


In [84]:
for x, ll in lks.items():
    lks[x] = [ x for x,y in zip(sites,ll) if y >0]

In [71]:
lks = {'X_lzx.html': ['X_ask.html', 'X_hvc.html', 'X_lnh.html', 'X_dbo.html'],
 'X_ask.html': ['X_lzx.html',
  'X_ask.html',
  'X_hvc.html',
  'X_lnh.html',
  'X_cgg.html',
  'X_ifl.html'],
 'X_hvc.html': ['X_hvc.html', 'X_lnh.html', 'X_cgg.html', 'X_dbo.html'],
 'X_lnh.html': ['X_hvc.html', 'X_cgg.html', 'X_ifl.html'],
 'X_cgg.html': ['X_ask.html', 'X_hvc.html', 'X_lnh.html', 'X_dbo.html'],
 'X_dbo.html': ['X_ask.html', 'X_hvc.html', 'X_lnh.html', 'X_ifl.html'],
 'X_ifl.html': ['X_lzx.html', 'X_hvc.html', 'X_lnh.html', 'X_cgg.html']}

In [66]:
for x, ll in lks.items():
    with open(x,'w') as fp:
        
        fp.write('# ' + x + '<br>\n'*3 )
        body = ''.join(['<a href="./{}"> {}</a> <br>\n'.format(y,y) for  y in ll])
        fp.write(body)

In [68]:
! ../.g

[master 6d7c30b] web
 1 file changed, 136 insertions(+), 7 deletions(-)
Counting objects: 4, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 2.92 KiB | 2.92 MiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/macbuse/macbuse.github.io.git
   78de9ae..6d7c30b  master -> master


In [92]:
base = 'https://macbuse.github.io/PROG/'
root = 'X_hvc.html'

In [1]:
import requests
import re

In [80]:
r = requests.get(root)

In [81]:
r.text

'# X_hvc.html<br>\n<br>\n<br>\n<a href="./X_hvc.html"> X_hvc.html</a> <br>\n<a href="./X_lnh.html"> X_lnh.html</a> <br>\n<a href="./X_cgg.html"> X_cgg.html</a> <br>\n<a href="./X_dbo.html"> X_dbo.html</a> <br>\n'

['./X_hvc.html', './X_lnh.html', './X_cgg.html', './X_dbo.html']

In [28]:
import requests
import re

pax = re.compile('<a href="(.*?)">', re.DOTALL)

base = 'https://macbuse.github.io/PROG/'
root = 'X_hvc.html'

todo = [root]
done, nodes = [], {}

while todo:
    parent = todo.pop()
    if parent in done: continue
    done.append(parent)
    r = requests.get(base + parent)
    children = [ x[2:] for x in pax.findall(r.text)]
    nodes[parent] = children

    todo.extend(children)
    todo = list(set(todo))

k

7

In [27]:
nodes

{'X_hvc.html': ['X_hvc.html', 'X_lnh.html', 'X_cgg.html', 'X_dbo.html'],
 'X_cgg.html': ['X_ask.html', 'X_hvc.html', 'X_lnh.html', 'X_dbo.html'],
 'X_dbo.html': ['X_ask.html', 'X_hvc.html', 'X_lnh.html', 'X_ifl.html'],
 'X_ask.html': ['X_lzx.html',
  'X_ask.html',
  'X_hvc.html',
  'X_lnh.html',
  'X_cgg.html',
  'X_ifl.html'],
 'X_ifl.html': ['X_lzx.html', 'X_hvc.html', 'X_lnh.html', 'X_cgg.html'],
 'X_lnh.html': ['X_hvc.html', 'X_cgg.html', 'X_ifl.html'],
 'X_lzx.html': ['X_ask.html', 'X_hvc.html', 'X_lnh.html', 'X_dbo.html']}

In [35]:
import numpy as np

In [36]:
pages = list(nodes.keys())

M = np.zeros((7,7))

for k, p in enumerate(pages):
    M[k, [pages.index(x) for x in nodes[p] ] ]  = 1

In [37]:
M

array([[1., 1., 1., 0., 0., 1., 0.],
       [1., 0., 1., 1., 0., 1., 0.],
       [1., 0., 0., 1., 1., 1., 0.],
       [1., 1., 0., 1., 1., 1., 1.],
       [1., 1., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 1., 0., 0.],
       [1., 0., 1., 1., 0., 1., 0.]])

In [14]:
lks.keys()

dict_keys(['X_lzx.html', 'X_ask.html', 'X_hvc.html', 'X_lnh.html', 'X_cgg.html', 'X_dbo.html', 'X_ifl.html'])

In [15]:
nodes.keys()

dict_keys(['X_hvc.html', 'X_cgg.html', 'X_dbo.html', 'X_ask.html', 'X_ifl.html', 'X_lnh.html'])

In [16]:
nodes['X_lzx.html']

KeyError: 'X_lzx.html'

In [57]:
import time
import urllib

import bs4
import requests


start_url = "https://en.wikipedia.org/wiki/Special:Random"
target_url = "https://en.wikipedia.org/wiki/Narendra_Modi"

pp = re.compile('<p>(.*?)</p>', re.DOTALL)
pax = re.compile(r'<a href="(.*?)"', re.DOTALL)

def find_first(url):
    response = requests.get(url)
    html = response.text
    
    paras = pp.findall(html)
    if not paras: return
    
    lks = pax.findall(''.join(paras))
    if not lks: return
    
    article_link = lks[0]

    first_link = urllib.parse.urljoin('https://en.wikipedia.org/', article_link)
    return first_link

def continue_crawl(search_history, target_url, max_steps=25):
    if search_history[-1] == target_url:
        print("We've found the target article!")
        return False
    elif len(search_history) > max_steps:
        print("The search has gone on suspiciously long, aborting search!")
        return False
    elif search_history[-1] in search_history[:-1]:
        print("We've arrived at an article we've already seen, aborting search!")
        return False
    else:
        return True

article_chain = [start_url]

while continue_crawl(article_chain, target_url):
    print(article_chain[-1])

    first_link = find_first(article_chain[-1])
    if not first_link:
        print("We've arrived at an article with no links, aborting search!")
        break

    article_chain.append(first_link)

    time.sleep(2) # slow down otherwise wiki server will block you

https://en.wikipedia.org/wiki/Special:Random
https://en.wikipedia.org/wiki/County_Championship
https://en.wikipedia.org/wiki/First-class_cricket
https://en.wikipedia.org/wiki/Cricket
https://en.wikipedia.org/wiki/Bat-and-ball_games
https://en.wikipedia.org/#cite_note-1
https://en.wikipedia.org/wiki/Dragon:_The_Bruce_Lee_Story
https://en.wikipedia.org/wiki/Biographical_film
https://en.wikipedia.org/wiki/Help:IPA/English
https://en.wikipedia.org/wiki/International_Phonetic_Alphabet
https://en.wikipedia.org/wiki/Alphabet
https://en.wikipedia.org/wiki/Symbols
https://en.wikipedia.org/wiki/Word
https://en.wikipedia.org/wiki/Linguistics
https://en.wikipedia.org/wiki/Science
https://en.wikipedia.org/wiki/Latin
https://en.wikipedia.org/wiki/Help:IPA/Latin
We've arrived at an article we've already seen, aborting search!


In [40]:
url = start_url

response = requests.get(url)
html = response.text

In [51]:
txt = ''.join(pp.findall(html))
pax.findall(txt)

['/wiki/Vine',
 '/wiki/Apocynaceae',
 '/wiki/Indomalaya',
 '/wiki/Australasia',
 '/wiki/Melanesia',
 '#cite_note-u-1',
 '/wiki/Leaf',
 '#cite_note-NSW-2',
 '/wiki/Latex',
 '#cite_note-NSW-2',
 '#cite_note-NSW-2',
 '/wiki/Robert_Brown_(botanist,_born_1773)',
 '/wiki/Wernerian_Natural_History_Society',
 '#cite_note-APNI-3',
 '/wiki/James_Parsons_(physician)',
 '/wiki/Royal_Society',
 '#cite_note-4',
 '/wiki/Conserved_name',
 '/wiki/Homonym',
 '/wiki/Patrick_Browne',
 '/wiki/Lythraceae',
 '/wiki/Cuphea',
 '#cite_note-GRIN-5',
 '/wiki/Artia_(plant)',
 '/wiki/Prestonia_(plant)',
 '#cite_note-6',
 '#cite_note-7',
 '#cite_note-u-1',
 '#cite_note-18',
 '#cite_note-FONZ-11']

In [59]:
! ../.g

[master 96adb8d] web
 2 files changed, 427 insertions(+), 44 deletions(-)
Counting objects: 5, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 3.49 KiB | 3.49 MiB/s, done.
Total 5 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/macbuse/macbuse.github.io.git
   6d7c30b..96adb8d  master -> master
